In [1]:
import pandas as pd

In [117]:
file = 'train_data.csv'
area_total = 2
limite_agua = 5000
umidade_solo = 0
janela_dias = 90
orcamento_maximo = 0

In [66]:
df = pd.read_csv(file)

In [11]:
m2_area = area_total * 10_000

In [67]:
df.head()

,CULTURA,CUSTO PRODUÇÃO,PRODUTIVIDADE,CICLO DE VIDA MIN EM DIAS,CICLO DE VIDA MAX EM DIAS,REQUISITO DE ÁGUA,RETORNO FINANCEIRO,SINERGIA,COMPANHEIRA,NEUTRA,ANTAGÔNICA,ESPAÇO MÍNIMO m²
0,milho,2.0,3.0,90,120,2.0,2.5,feijão,"mandioca,abóbora","arroz,trigo","algodão,cana-de-açúcar",0.7
1,soja,2.5,3.0,90,150,2.0,3.0,sorgo,"milho,trigo","feijão,arroz","café,cana-de-açúcar",0.4
2,café,3.0,2.0,1096,1461,3.0,3.0,feijão,"milho,banana","mandioca,arroz","soja,cana-de-açúcar",2.5
3,feijão,1.5,2.0,60,90,2.0,2.0,milho,"mandioca,batata","soja,trigo","algodão,cana-de-açúcar",0.4
4,sorgo,1.0,2.5,90,130,1.0,2.0,soja,"milho,trigo","feijão,mandioca","arroz,café",0.5


Usar codificação hibrida

objetivo maior produtividade com menor custo

fit = produtividade + ret. fin - cust. prod

In [16]:
df['CULTURA'][0]

'milho'

In [19]:
fitness = df['CULTURA'].str.split(' ', expand=True)

In [137]:
sample = df.sample(3)

# pontuação normalizada do conjunto
W_prod = 0.6
W_comp = 0.4

# para cada cultura do conjunto
# pontuação_produtividade = (produtividade/espaço minimo)
normalized_production_score = 0
normalized_competition_score = 0

for s in sample.iterrows():
    production_score = (s[1]['PRODUTIVIDADE'] / s[1]['ESPAÇO MÍNIMO m²'])
    normalized_production_score += production_score


# para cada cultura do conjunto
# pontuação_competitividade = 0
# +1.0 Sinérgica
# +0.5 Companheira
# 0.0 Neutra
# -1.0 Antagônica

plant_map = dict()

for s in sample.iterrows():
    plant_map[s[1]['CULTURA']] = dict(sinergia=s[1]['SINERGIA'].split(','),companheira=s[1]['COMPANHEIRA'].split(','),neutra=s[1]['NEUTRA'].split(','),antagonica=s[1]['ANTAGÔNICA'].split(','))

# print(plant_map)

#score de compatibilidade
for s in sample.iterrows():
    for key, value in plant_map.items():
        if s[1]['CULTURA'] == key:
            continue
        if s[1]['CULTURA'] in value['sinergia']:
            normalized_competition_score += 1.0
        if s[1]['CULTURA'] in value['companheira']:
            normalized_competition_score += 0.5
        if s[1]['CULTURA'] in value['neutra']:
            normalized_competition_score += 0.0
        if s[1]['CULTURA'] in value['antagonica']:
            normalized_competition_score -= 1.0


# normalizar para ficar entre 0 e 1
print(normalized_competition_score)


# score = (W_prod * production_score)+(W_comp * competition_score)
score = (W_prod * normalized_production_score)+(W_comp * normalized_competition_score)

print(score)

# penalidades

custo_total = 0
agua_total = 0
ciclo_de_vida = []

for s in sample.iterrows():
    custo_total += s[1]['CUSTO PRODUÇÃO']
    agua_total += s[1]['REQUISITO DE ÁGUA']
    ciclo_de_vida.append(s[1]['CICLO DE VIDA MAX EM DIAS'])

#Se Custo Total > Orçamento_Máximo, então P_econômica = 0.01, senão P_econômica=1

P_economica = 0.01 if custo_total >= orcamento_maximo else 1

# Água Total > Sua_Disponibilidade_de_Água, então P_ecológica = 0.01, senão P_ecológica = 1

P_ecologica = 0.01 if agua_total >= limite_agua else 1

# ciclo de vida: verificar se o ciclo de vida mais longo cabe na janela de plantio

if max(ciclo_de_vida) > janela_dias:
    P_ecologica += 0.5

fitness = score * P_economica * P_ecologica

print(fitness)

1.0
16.38
0.2457
